In [ ]:
from control import lqr
from common_function import newmark, eigenvalue
import numpy as np
from matplotlib import pyplot as plt
import os, re
import pandas as pd

In [ ]:
m = 20  # 单位：kg
k = 2700  # 单位：N/m
M = np.array([[m]])
K = np.array([[k]])
zeta = 0.02  # 结构固有阻尼比

w, T, phi, gamma, M_par = eigenvalue(M, K)
w0 = w[0]

In [ ]:
c = 2 * zeta * m * w0

In [ ]:
mu = 0.0761
ksi = 0.0072
kpa = 0.0791
md0 = mu * m
cd0 = 2 * ksi * m * w0
kd0 = kpa * k

In [ ]:
# 半主动惯容基本参数
min_max = md0
min_min = 0
cd_max = cd0
cd_min = 0
kpa = 10
kd = kpa * k

In [ ]:
# 状态空间矩阵
A = np.array([[0, 1], [-w0 ** 2, -2 * zeta * w0]], dtype=float)
B = np.array([[0, 0], [0, -1 / m]], dtype=float)

In [ ]:
wave = 0
dt = 0
name = "white_noise_190117_3.txt"
root = "./waves"
print(name)
wave_name = name.split('.')[0]
if name.endswith(".AT2"):
    filename = os.path.join(root, name)
    with open(filename, 'r') as fo:
        for i in range(3):
            fo.readline()
        line = fo.readline()
        target = re.search(r'DT= *(.*) *SEC', line, re.I | re.M)
        dt = float(target.group(1))
        wave = pd.read_table(filename, skiprows=4, sep="  ", header=None, engine='python')
        wave = wave.fillna(0)
        wave = wave.values
        size = wave.size
        wave = wave.reshape((size,))
if name.endswith(".txt"):
    filename = os.path.join(root, name)
    wave = np.loadtxt(filename)
    dt = 0.02
wave = wave / max(abs(wave)) * 0.3 * 9.8
wave_len = len(wave)
time = np.arange(0, wave_len * dt, dt)

In [ ]:
# 加速度插值
dt_s = 0.001
t_s = np.arange(0, max(time), dt_s)
wave_s = np.interp(t_s, time, wave)
wave_len_s = len(wave_s)

In [ ]:
wave_s

In [ ]:
# 原结构
u0 = np.zeros([2, wave_len_s])
du0 = np.zeros([2, wave_len_s])
du0[1, 0] = -wave_s[0]
for i in range(0, wave_len_s - 1):
    p = np.array([[0], [-wave_s[i]]])
    du0[..., i + 1, None] = np.dot(A, u0[..., i, None]) + p
    u0[..., i + 1] = u0[..., i] + du0[..., i + 1] * dt_s

In [ ]:
M0 = np.array(m, ndmin=2, dtype=float)
C0 = np.array(c, ndmin=2, dtype=float)
K0 = np.array(k, ndmin=2, dtype=float)
E0 = np.array(1, ndmin=2, dtype=float)
y, dy, ddy, ddy_ab = newmark(K0, M0, C0, wave_s, dt, E0)

In [ ]:
plt.plot(t_s, u0[0, :].reshape(-1))
plt.plot(t_s, y.reshape(-1))

In [ ]:
# 被动控制结构
M = np.array([[m, 0], [0, md0]], ndmin=2, dtype=float)
K = np.array([[k + kd0, -kd0], [-kd0, kd0]], ndmin=2, dtype=float)
C = np.array([[c, 0], [0, cd0]], ndmin=2, dtype=float)
E = np.array([[1], [0]], ndmin=2, dtype=float)
u1, du1, _, ddu1_ab = newmark(K, M, C, wave_s, dt_s, E)

In [ ]:
plt.plot(t_s, u0[0, :].reshape(-1))
plt.plot(t_s, u1[0, :])

In [ ]:
# 反馈增益矩阵
Q = np.diag([100, 0])
R = np.diag([1e-7, 1e-7])
D, _, _ = lqr(A, B, Q, R)
D = -D

In [ ]:
# 半主动控制结构
u2 = np.zeros([2, wave_len_s])
du2 = np.zeros([2, wave_len_s])
du2[1, 0] = -wave_s[0]
F = np.zeros([2, wave_len_s])
ft = np.zeros(wave_len_s)
ud = np.zeros(wave_len_s)
dud = np.zeros(wave_len_s)
ddud = np.zeros(wave_len_s)
fmax = np.zeros(wave_len_s)
fmin = np.zeros(wave_len_s)
fa = np.zeros(wave_len_s)

ud_pre = 0
dud_pre = 0

for i in range(wave_len_s - 1):
    p = np.array([[0], [-wave_s[i]]])  # 当前荷载向量
    F[:, i, None] = np.dot(D, u2[:, i, None])  # 当前反馈力
    ft[i] = F[1, i]  # 当前目标出力
    ud[i] = u2[0, i] - ft[i] / kd  # 当前惯容位移
    dud[i] = (ud[i] - ud_pre) / dt_s  # 当前惯容速度
    ddud[i] = (dud[i] - dud_pre) / dt_s  # 当前惯容加速度
    ud_pre = ud[i]
    dud_pre = dud[i]

    # 当前出力上限
    fmax[i] = max(
        [min_max * ddud[i] + cd_max * dud[i],
         min_max * ddud[i] + cd_min * dud[i],
         min_min * ddud[i] + cd_max * dud[i],
         min_min * ddud[i] + cd_min * dud[i]])
    # 当前出力下限
    fmin[i] = min([min_max * ddud[i] + cd_max * dud[i],
                   min_max * ddud[i] + cd_min * dud[i],
                   min_min * ddud[i] + cd_max * dud[i],
                   min_min * ddud[i] + cd_min * dud[i]])
    if ft[i] < fmin[i]:
        fa[i] = fmin[i]
    elif ft[i] > fmax[i]:
        fa[i] = fmax[i]
    else:
        fa[i] = ft[i]
    F[1, i] = fa[i]
    du2[:, i + 1, None] = np.dot(A, u2[..., i, None]) + np.dot(B, F[..., i, None])  + p
    u2[..., i + 1] = u2[..., i] + du2[..., i + 1] * dt_s

In [ ]:
plt.plot(t_s, u0[0, :].reshape(-1))
plt.plot(t_s, u1[0, :])
plt.plot(t_s, u2[0, :])

In [ ]:
plt.plot(t_s[0: 100], fmax[0: 100])
plt.plot(t_s[0: 100], fmin[0: 100])
plt.plot(t_s[0: 100], fa[0: 100])

In [ ]:
plt.plot(t_s, ud)

In [ ]:
y = u2[0, :]
dy = u2[1, :]
ddy_ab = du2[1, :]

In [ ]:
length = len(y)
numpy_data = np.zeros([length, 9])
data = pd.DataFrame(data=numpy_data, columns=["str_cur_disp", "str_cur_vel", "str_cur_acc", "str_pre_disp", "str_pre_vel", "str_pre_acc", "ft_pre", "ft_cur", "ft_next"])

In [ ]:
# 构造样本数据

for i in range(1, length - 1):
    row_data = np.array([y[i], dy[i], ddy_ab[i], y[i - 1], dy[i - 1], ddy_ab[i - 1], ft[i - 1], ft[i], ft[i + 1]])
    data.loc[i - 1] = row_data

In [ ]:
x_label = ["str_cur_disp", "str_cur_vel", "str_cur_acc", "str_pre_disp", "str_pre_vel", "str_pre_acc", "ft_pre", "ft_cur"]
y_label = "ft_next"
X = data.loc[:, x_label].values
y = data.loc[:, y_label].values

In [ ]:
data_len = data.shape[0]
train_len = int(0.6 * data_len)
X_train = X[0: train_len, :]
X_test = X[train_len: data_len, :]
y_train = y[0: train_len]
y_test = y[train_len: data_len]

In [ ]:
# 特征标准化，采用最大最小值标准化，转化后的值范围（0,1）
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
new_X_train = X_train
new_X_test = X_test
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
# normalizer = Normalizer(copy=True, norm='l2').fit(new_X_train)
# new_X_train = normalizer.transform(new_X_train)
# new_X_test = normalizer.transform(new_X_test)
ss = StandardScaler().fit(new_X_train)
new_X_train = ss.transform(new_X_train)
new_X_test = ss.transform(new_X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
def plot_regression_results(ax, y_true, y_pred, title, scores):
    """预测目标与真实目标的散点图。"""
    ax.plot([y_true.min(), y_true.max()],
            [y_true.min(), y_true.max()],'--r', linewidth=2)
    ax.scatter(y_true, y_pred, alpha=0.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    ax.set_xlim([y_true.min(), y_true.max()])
    ax.set_ylim([y_true.min(), y_true.max()])
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    extra = plt.Rectangle((0, 0), 0, 0, fc="w", fill=False,
                          edgecolor='none', linewidth=0)
    ax.legend([extra], [scores], loc='upper left')
    ax.set_title(title)


def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
GBoost.fit(new_X_train, y_train)


train_pred = GBoost.predict(new_X_train)
pred = GBoost.predict(new_X_test)
print(rmsle(y_train, train_pred))

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
r2 = r2_score(y_test, pred)
mse = mean_squared_error(y_test, pred)
mape = mean_absolute_percentage_error(y_test, pred)

plt.figure(figsize=(10,8))
ax = plt.gca()
plot_regression_results(
        ax, y_test, pred,
        "Stacking Regressor",
        (r'$R^2={:.2f}$' + '\n' + r'$MAPE={:.2f}$' + '\n' + r'$MSE={:.2f}$')
        .format(r2, mse, mape))


In [ ]:
# 半主动控制结构
u2_ml = np.zeros([2, wave_len_s])
du2_ml = np.zeros([2, wave_len_s])
du2_ml[1, 0] = -wave_s[0]
F = np.zeros([2, wave_len_s])
ft_ml = np.zeros(wave_len_s)
ud_ml = np.zeros(wave_len_s)
dud_ml = np.zeros(wave_len_s)
ddud_ml = np.zeros(wave_len_s)
fmax_ml = np.zeros(wave_len_s)
fmin_ml = np.zeros(wave_len_s)
fa_ml = np.zeros(wave_len_s)
ud_ml_pre = 0
dud_ml_pre = 0

str_pre_disp = 0
str_pre_vel = 0
str_pre_acc = 0
str_cur_disp = 0
str_cur_vel = 0
str_cur_acc = 0
ft_ml_pre = 0
ft_ml_next = 0
for i in range(1, wave_len_s - 1):
    p = np.array([[0], [-wave_s[i]]])  # 当前荷载向量
    str_cur_disp = u2_ml[0, i - 1]
    str_cur_vel = u2_ml[1, i - 1]
    str_cur_acc = du2_ml[1, i - 1]
    ft_ml_cur = F[1, i - 1]
    x = np.array([str_cur_disp, str_cur_vel, str_cur_acc, str_pre_disp, str_pre_vel, str_pre_acc, ft_ml_pre, ft_ml_cur], ndmin=2)
    x = ss.transform(x)
    ft_ml[i] = GBoost.predict(x)[0]
    ud_ml[i] = u2_ml[0, i] - ft_ml[i] / kd  # 当前惯容位移
    dud_ml[i] = (ud_ml[i] - ud_ml_pre) / dt_s  # 当前惯容速度
    ddud_ml[i] = (dud_ml[i] - dud_ml_pre) / dt_s  # 当前惯容加速度
    ud_ml_pre = ud_ml[i]
    dud_ml_pre = dud_ml[i]
    # 当前出力上限
    fmax_ml[i] = max(
        [min_max * ddud_ml[i] + cd_max * dud_ml[i],
         min_max * ddud_ml[i] + cd_min * dud_ml[i],
         min_min * ddud_ml[i] + cd_max * dud_ml[i],
         min_min * ddud_ml[i] + cd_min * dud_ml[i]])
    # 当前出力下限
    fmin_ml[i] = min([min_max * ddud_ml[i] + cd_max * dud_ml[i],
                   min_max * ddud_ml[i] + cd_min * dud_ml[i],
                   min_min * ddud_ml[i] + cd_max * dud_ml[i],
                   min_min * ddud_ml[i] + cd_min * dud_ml[i]])
    if ft_ml[i] < fmin_ml[i]:
        fa_ml[i] = fmin_ml[i]
    elif ft_ml[i] > fmax_ml[i]:
        fa_ml[i] = fmax_ml[i]
    else:
        fa_ml[i] = ft_ml[i]
    F[1, i] = fa_ml[i]
    ft_ml_pre = ft_ml_cur
    str_pre_disp = str_cur_disp
    str_pre_vel = str_cur_vel
    str_pre_acc = str_cur_acc
    du2_ml[:, i + 1, None] = np.dot(A, u2_ml[..., i, None]) + np.dot(B, F[..., i, None])  + p
    u2_ml[..., i + 1] = u2_ml[..., i] + du2_ml[..., i + 1] * dt_s

In [ ]:
plt.plot(t_s, u0[0, :].reshape(-1))
plt.plot(t_s, u1[0, :])
plt.plot(t_s, u2[0, :])
plt.plot(t_s, u2_ml[0, :])

In [ ]:
plt.plot(t_s, fa)
plt.plot(t_s, fa_ml)

In [ ]:
# 用另外一条波
wave = 0
dt = 0
name = "RSN93_SFERN_WND143.AT2"
root = "./waves"
print(name)
wave_name = name.split('.')[0]
if name.endswith(".AT2"):
    filename = os.path.join(root, name)
    with open(filename, 'r') as fo:
        for i in range(3):
            fo.readline()
        line = fo.readline()
        target = re.search(r'DT= *(.*) *SEC', line, re.I | re.M)
        dt = float(target.group(1))
        wave = pd.read_table(filename, skiprows=4, sep="  ", header=None, engine='python')
        wave = wave.fillna(0)
        wave = wave.values
        size = wave.size
        wave = wave.reshape((size,))
if name.endswith(".txt"):
    filename = os.path.join(root, name)
    wave = np.loadtxt(filename)
    dt = 0.02
wave = wave * 0.3 * 9.8
wave_len = len(wave)
time = np.arange(0, wave_len * dt, dt)

In [ ]:
# 加速度插值
dt_s = 0.001
t_s = np.arange(0, max(time), dt_s)
wave_s = np.interp(t_s, time, wave)
wave_len_s = len(wave_s)

In [ ]:
# 原结构
u0 = np.zeros([2, wave_len_s])
du0 = np.zeros([2, wave_len_s])
du0[1, 0] = -wave_s[0]
for i in range(0, wave_len_s - 1):
    p = np.array([[0], [-wave_s[i]]])
    du0[..., i + 1, None] = np.dot(A, u0[..., i, None]) + p
    u0[..., i + 1] = u0[..., i] + du0[..., i + 1] * dt_s

In [ ]:
# 被动控制结构
M = np.array([[m, 0], [0, md0]], ndmin=2, dtype=float)
K = np.array([[k + kd0, -kd0], [-kd0, kd0]], ndmin=2, dtype=float)
C = np.array([[c, 0], [0, cd0]], ndmin=2, dtype=float)
E = np.array([[1], [0]], ndmin=2, dtype=float)
u1, du1, _, ddu1_ab = newmark(K, M, C, wave_s, dt_s, E)

In [ ]:
plt.plot(t_s, u0[0, :].reshape(-1))
plt.plot(t_s, u1[0, :])

In [ ]:
# 半主动控制结构
u2 = np.zeros([2, wave_len_s])
du2 = np.zeros([2, wave_len_s])
du2[1, 0] = -wave_s[0]
F = np.zeros([2, wave_len_s])
ft = np.zeros(wave_len_s)
ud = np.zeros(wave_len_s)
dud = np.zeros(wave_len_s)
ddud = np.zeros(wave_len_s)
fmax = np.zeros(wave_len_s)
fmin = np.zeros(wave_len_s)
fa = np.zeros(wave_len_s)

ud_pre = 0
dud_pre = 0

for i in range(wave_len_s - 1):
    p = np.array([[0], [-wave_s[i]]])  # 当前荷载向量
    F[:, i, None] = np.dot(D, u2[:, i, None])  # 当前反馈力
    ft[i] = F[1, i]  # 当前目标出力
    ud[i] = u2[0, i] - ft[i] / kd  # 当前惯容位移
    dud[i] = (ud[i] - ud_pre) / dt_s  # 当前惯容速度
    ddud[i] = (dud[i] - dud_pre) / dt_s  # 当前惯容加速度
    ud_pre = ud[i]
    dud_pre = dud[i]

    # 当前出力上限
    fmax[i] = max(
        [min_max * ddud[i] + cd_max * dud[i],
         min_max * ddud[i] + cd_min * dud[i],
         min_min * ddud[i] + cd_max * dud[i],
         min_min * ddud[i] + cd_min * dud[i]])
    # 当前出力下限
    fmin[i] = min([min_max * ddud[i] + cd_max * dud[i],
                   min_max * ddud[i] + cd_min * dud[i],
                   min_min * ddud[i] + cd_max * dud[i],
                   min_min * ddud[i] + cd_min * dud[i]])
    if ft[i] < fmin[i]:
        fa[i] = fmin[i]
    elif ft[i] > fmax[i]:
        fa[i] = fmax[i]
    else:
        fa[i] = ft[i]
    F[1, i] = fa[i]
    du2[:, i + 1, None] = np.dot(A, u2[..., i, None]) + np.dot(B, F[..., i, None])  + p
    u2[..., i + 1] = u2[..., i] + du2[..., i + 1] * dt_s

In [ ]:
plt.plot(t_s, u0[0, :].reshape(-1))
plt.plot(t_s, u1[0, :])
plt.plot(t_s, u2[0, :])

In [ ]:
# 半主动控制结构
u2_ml = np.zeros([2, wave_len_s])
du2_ml = np.zeros([2, wave_len_s])
du2_ml[1, 0] = -wave_s[0]
F = np.zeros([2, wave_len_s])
ft_ml = np.zeros(wave_len_s)
ud_ml = np.zeros(wave_len_s)
dud_ml = np.zeros(wave_len_s)
ddud_ml = np.zeros(wave_len_s)
fmax_ml = np.zeros(wave_len_s)
fmin_ml = np.zeros(wave_len_s)
fa_ml = np.zeros(wave_len_s)
ud_ml_pre = 0
dud_ml_pre = 0

str_pre_disp = 0
str_pre_vel = 0
str_pre_acc = 0
str_cur_disp = 0
str_cur_vel = 0
str_cur_acc = 0
ft_ml_pre = 0
ft_ml_next = 0
for i in range(1, wave_len_s - 1):
    p = np.array([[0], [-wave_s[i]]])  # 当前荷载向量
    str_cur_disp = u2_ml[0, i - 1]
    str_cur_vel = u2_ml[1, i - 1]
    str_cur_acc = du2_ml[1, i - 1]
    ft_ml_cur = F[1, i - 1]
    x = np.array([str_cur_disp, str_cur_vel, str_cur_acc, str_pre_disp, str_pre_vel, str_pre_acc, ft_ml_pre, ft_ml_cur], ndmin=2)
    x = ss.transform(x)
    ft_ml[i] = GBoost.predict(x)[0]
    ud_ml[i] = u2_ml[0, i] - ft_ml[i] / kd  # 当前惯容位移
    dud_ml[i] = (ud_ml[i] - ud_ml_pre) / dt_s  # 当前惯容速度
    ddud_ml[i] = (dud_ml[i] - dud_ml_pre) / dt_s  # 当前惯容加速度
    ud_ml_pre = ud_ml[i]
    dud_ml_pre = dud_ml[i]
    # 当前出力上限
    fmax_ml[i] = max(
        [min_max * ddud_ml[i] + cd_max * dud_ml[i],
         min_max * ddud_ml[i] + cd_min * dud_ml[i],
         min_min * ddud_ml[i] + cd_max * dud_ml[i],
         min_min * ddud_ml[i] + cd_min * dud_ml[i]])
    # 当前出力下限
    fmin_ml[i] = min([min_max * ddud_ml[i] + cd_max * dud_ml[i],
                   min_max * ddud_ml[i] + cd_min * dud_ml[i],
                   min_min * ddud_ml[i] + cd_max * dud_ml[i],
                   min_min * ddud_ml[i] + cd_min * dud_ml[i]])
    if ft_ml[i] < fmin_ml[i]:
        fa_ml[i] = fmin_ml[i]
    elif ft_ml[i] > fmax_ml[i]:
        fa_ml[i] = fmax_ml[i]
    else:
        fa_ml[i] = ft_ml[i]
    F[1, i] = fa_ml[i]
    ft_ml_pre = ft_ml_cur
    str_pre_disp = str_cur_disp
    str_pre_vel = str_cur_vel
    str_pre_acc = str_cur_acc
    du2_ml[:, i + 1, None] = np.dot(A, u2_ml[..., i, None]) + np.dot(B, F[..., i, None])  + p
    u2_ml[..., i + 1] = u2_ml[..., i] + du2_ml[..., i + 1] * dt_s

In [ ]:
plt.plot(t_s, u0[0, :].reshape(-1))
plt.plot(t_s, u1[0, :])
plt.plot(t_s, u2[0, :])
plt.plot(t_s, u2_ml[0, :])

In [ ]:
plt.plot(t_s, ddud)
plt.plot(t_s, ddud_ml)

In [ ]:
plt.plot(t_s, fmax)
plt.plot(t_s, fmin)
plt.plot(t_s, fa)

In [ ]:
plt.plot(t_s, fmax_ml)
plt.plot(t_s, fmin_ml)
plt.plot(t_s, fa_ml)

In [ ]:
plt.plot(t_s, fmax)
plt.plot(t_s, fmin)
plt.plot(t_s, ft)

In [ ]:
plt.plot(t_s, ft)
plt.plot(t_s, ft_ml)